In [2]:
# -*- coding=utf-8 -*-
import pandas as pd
from association_measures import frequencies as fq

from source.utils.dataframes import (corners, drop_margins, print_md_table,
                                     set_axes, square_sample)
from source.utils.general import timestamp_now as tnow

WITH_SUM = True
FRQ_TABLE_CSV = '/home/arh234/projects/sanpi/results/freq_out/RBXadj/all_adj-x-adv_frq-thr0-001p.35f=868+.csv'

df = set_axes(pd.read_csv(FRQ_TABLE_CSV))
corners(df)

adv_form_lower,SUM,very,more,most,so,...,on,emphatically,second-most,cosmetically,pointedly
adj_form_lower,,,,,,,,,,,
SUM,83284343,9913432,9320997,7568812,5735964,...,872,872,869,869,868
many,2210387,21237,373,140,1191864,...,1,3,0,0,0
important,2199447,359610,306604,748533,105509,...,0,2,105,9,0
good,2030480,507499,18902,5207,153196,...,1,6,0,7,0
much,1776924,42365,102,22,614652,...,95,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
untrained,872,9,10,43,11,...,0,0,0,0,0
carnal,872,34,150,62,27,...,0,0,0,0,0
contiguous,871,4,106,8,6,...,0,0,0,0,0


In [25]:
ex_3x3 = square_sample(
    (df
     .loc[~df.index.isin(['much', 'most', 'good', 'less', 'least', 'best', 'more', 'own', 'worth']),
          ~df.columns.isin(['very', 'long', 'plain', 'once', 'now', 'not', "n't", 'no',
                            'then', 'just', 'all', 'about', 'as', 'so', 'well', 'way',
                            'almost', 'mostly', 'most', 'much', 'more'])
          ]
     .filter(regex=r'[^e]r|[^r]$', axis=0)
     .iloc[:200, :150]),
    n=3, with_margin=WITH_SUM)

ex_3x3.to_csv('/share/compling/projects/sanpi/info/samples/'
              f"freq_3x3{'.with-sum' if WITH_SUM else ''}.{tnow()}.csv")
ex_3x3

adv_form_lower,SUM,nearly,fundamentally,extremely
adj_form_lower,,,,
SUM,83284343,174561,40509,976163
critical,141953,12,46,2130
affordable,126073,8,2,2253
healthy,113066,14,114,1223


In [26]:
print_ex = ex_3x3.copy()
print_ex.index.name = None
print_ex.index = '***' + print_ex.index + '***'
print_ex.columns = '*' + print_ex.columns + '*'
print(print_md_table(print_ex.rename(
    columns={'*SUM*': '`SUM`'}, index={'***SUM***': '**`SUM`**'}), suppress=True))

|                  |      `SUM` |   *nearly* |   *fundamentally* |   *extremely* |
|:-----------------|-----------:|-----------:|------------------:|--------------:|
| **`SUM`**        | 83,284,343 |    174,561 |            40,509 |       976,163 |
| ***critical***   |    141,953 |         12 |                46 |         2,130 |
| ***affordable*** |    126,073 |          8 |                 2 |         2,253 |
| ***healthy***    |    113,066 |         14 |               114 |         1,223 |


|                  |      `SUM` |   *nearly* |   *fundamentally* |   *extremely* |
|:-----------------|-----------:|-----------:|------------------:|--------------:|
| **`SUM`**        | 83,284,343 |    174,561 |            40,509 |       976,163 |
| ***critical***   |    141,953 |         12 |                46 |         2,130 |
| ***affordable*** |    126,073 |          8 |                 2 |         2,253 |
| ***healthy***    |    113,066 |         14 |               114 |         1,223 |

In [28]:
N = ex_3x3.at['SUM', 'SUM']
N

83284343

In [29]:
adv_margins = f1 = ex_3x3.iloc[0, 1:].T.to_frame('f1').reset_index()
adv_margins

,adv_form_lower,f1
0,nearly,174561
1,fundamentally,40509
2,extremely,976163


In [30]:
adj_margins = f2 = ex_3x3.iloc[1:, 0].to_frame('f2').reset_index()
adj_margins

,adj_form_lower,f2
0,critical,141953
1,affordable,126073
2,healthy,113066


In [31]:
_ex_3x3 = drop_margins(ex_3x3.copy())
_ex_3x3

adv_form_lower,nearly,fundamentally,extremely
adj_form_lower,,,
critical,12,46,2130
affordable,8,2,2253
healthy,14,114,1223


In [32]:
_ex_3x3 = _ex_3x3.unstack().to_frame('f').reset_index()
_ex_3x3

,adv_form_lower,adj_form_lower,f
0,nearly,critical,12
1,nearly,affordable,8
2,nearly,healthy,14
3,fundamentally,critical,46
4,fundamentally,affordable,2
5,fundamentally,healthy,114
6,extremely,critical,2130
7,extremely,affordable,2253
8,extremely,healthy,1223


In [33]:
def merge_margins(_df, margins):
    margin_unit = margins.columns[0]
    return _df.merge(margins,
                    left_on=margin_unit,
                    right_on=margin_unit)


_ex_3x3 = merge_margins(merge_margins(_ex_3x3, f1), f2)
_ex_3x3

,adv_form_lower,adj_form_lower,f,f1,f2
0,nearly,critical,12,174561,141953
1,fundamentally,critical,46,40509,141953
2,extremely,critical,2130,976163,141953
3,nearly,affordable,8,174561,126073
4,fundamentally,affordable,2,40509,126073
5,extremely,affordable,2253,976163,126073
6,nearly,healthy,14,174561,113066
7,fundamentally,healthy,114,40509,113066
8,extremely,healthy,1223,976163,113066


In [34]:
_ex_3x3 = _ex_3x3.assign(
    bigram=_ex_3x3.iloc[:, 0] + '_' + _ex_3x3.iloc[:, 1],
    N=N)
_ex_3x3

,adv_form_lower,adj_form_lower,f,f1,f2,bigram,N
0,nearly,critical,12,174561,141953,nearly_critical,83284343
1,fundamentally,critical,46,40509,141953,fundamentally_critical,83284343
2,extremely,critical,2130,976163,141953,extremely_critical,83284343
3,nearly,affordable,8,174561,126073,nearly_affordable,83284343
4,fundamentally,affordable,2,40509,126073,fundamentally_affordable,83284343
5,extremely,affordable,2253,976163,126073,extremely_affordable,83284343
6,nearly,healthy,14,174561,113066,nearly_healthy,83284343
7,fundamentally,healthy,114,40509,113066,fundamentally_healthy,83284343
8,extremely,healthy,1223,976163,113066,extremely_healthy,83284343


In [35]:
full_frq_ex = _ex_3x3.join(fq.observed_frequencies(
    _ex_3x3)).join(fq.expected_frequencies(_ex_3x3))
full_frq_ex

,adv_form_lower,adj_form_lower,f,f1,f2,bigram,N,O11,O12,O21,O22,E11,E12,E21,E22
0,nearly,critical,12,174561,141953,nearly_critical,83284343,12,174549,141941,82967841,297.528404,174263.471596,141655.471596,8.296813e+07
1,fundamentally,critical,46,40509,141953,fundamentally_critical,83284343,46,40463,141907,83101927,69.045079,40439.954921,141883.954921,8.310195e+07
2,extremely,critical,2130,976163,141953,extremely_critical,83284343,2130,974033,139823,82168357,1663.809323,974499.190677,140289.190677,8.216789e+07
3,nearly,affordable,8,174561,126073,nearly_affordable,83284343,8,174553,126065,82983717,264.244493,174296.755507,125808.755507,8.298397e+07
4,fundamentally,affordable,2,40509,126073,fundamentally_affordable,83284343,2,40507,126071,83117763,61.321144,40447.678856,126011.678856,8.311782e+07
5,extremely,affordable,2253,976163,126073,extremely_affordable,83284343,2253,973910,123820,82184360,1477.682281,974685.317719,124595.317719,8.218358e+07
6,nearly,healthy,14,174561,113066,nearly_healthy,83284343,14,174547,113052,82996730,236.982286,174324.017714,112829.017714,8.299695e+07
7,fundamentally,healthy,114,40509,113066,fundamentally_healthy,83284343,114,40395,112952,83130882,54.994618,40454.005382,113011.005382,8.313082e+07
8,extremely,healthy,1223,976163,113066,extremely_healthy,83284343,1223,974940,111843,82196337,1325.229230,974837.770770,111740.770770,8.219644e+07


In [36]:
ex_row = full_frq_ex.sample(1).set_index('bigram')
ex_row.update(ex_row.select_dtypes('number').round(1))
ex_row.T

bigram,extremely_critical
adv_form_lower,extremely
adj_form_lower,critical
f,2130
f1,976163
f2,141953
N,83284343
O11,2130
O12,974033
O21,139823
O22,82168357


In [40]:
# def print_contingencies_for_bigram(frq_row: pd.Series or pd.DataFrame) -> None:
#     frq_row = frq_row.squeeze()

#     contingency_md = f'''
# ### Observed Contingencies for *{frq_row["adv_form_lower"]}* + *{frq_row["adj_form_lower"]}*

# |    | *{frq_row["adj_form_lower"]}* | $\\neg$ *{frq_row["adj_form_lower"]}* | across all `ADV` |
# |---:|---:|---:|---:|
# | ***{frq_row["adv_form_lower"]}*** | {frq_row["O11"]:,.0f} | {frq_row["O12"]:,.0f} | {frq_row["f1"]:,.0f} |
# | **$\\neg$ *{frq_row["adv_form_lower"]}*** | {frq_row["O21"]:,.0f} | {frq_row["O22"]:,.0f} | {frq_row["O21"] + frq_row["O22"]:,.0f} |
# | **across all `ADJ`** | {frq_row["f2"]:,.0f} | {frq_row["O12"] + frq_row["O22"]:,.0f} | {N:,.0f} |'''

#     print(contingency_md)

def print_contingencies_for_bigram(frq_row: pd.Series or pd.DataFrame) -> None:
    frq_row = frq_row.squeeze()
    adv_form_lower = frq_row["adv_form_lower"]
    adj_form_lower = frq_row["adj_form_lower"]
    O11 = frq_row["O11"]
    O12 = frq_row["O12"]
    O21 = frq_row["O21"]
    O22 = frq_row["O22"]
    adv_margin = frq_row["f1"]
    adj_margin = frq_row["f2"]
    adv_remainder = O21 + O22
    adj_remainder = O12 + O22

    contingency_md = f'''
### Observed Contingencies for *{adv_form_lower}* + *{adj_form_lower}*

|    | *{adj_form_lower}* | $\\neg$ *{adj_form_lower}* | across all `ADV` |
|---:|---:|---:|---:|
| ***{adv_form_lower}*** | {O11:,.0f} | {O12:,.0f} | {adv_margin:,.0f} |
| **$\\neg$ *{adv_form_lower}*** | {O21:,.0f} | {O22:,.0f} | {adv_remainder:,.0f} |
| **across all `ADJ`** | {adj_margin:,.0f} | {adj_remainder:,.0f} | {N:,.0f} |'''

    print(contingency_md)
    
print_contingencies_for_bigram(ex_row)


### Observed Contingencies for *extremely* + *critical*

|    | *critical* | $\neg$ *critical* | across all `ADV` |
|---:|---:|---:|---:|
| ***extremely*** | 2,130 | 974,033 | 976,163 |
| **$\neg$ *extremely*** | 139,823 | 82,168,357 | 82,308,180 |
| **across all `ADJ`** | 141,953 | 83,142,390 | 83,284,343 |


### Observed Contingencies for *extremely* + *critical*

|    | *critical* | $\neg$ *critical* | across all `ADV` |
|---:|---:|---:|---:|
| ***extremely*** | 2,130 | 974,033 | 976,163 |
| **$\neg$ *extremely*** | 139,823 | 82,168,357 | 82,308,180 |
| **across all `ADJ`** | 141,953 | 83,142,390 | 83,284,343 |



In [41]:
full_frq_ex.apply(print_contingencies_for_bigram, axis=1)


### Observed Contingencies for *nearly* + *critical*

|    | *critical* | $\neg$ *critical* | across all `ADV` |
|---:|---:|---:|---:|
| ***nearly*** | 12 | 174,549 | 174,561 |
| **$\neg$ *nearly*** | 141,941 | 82,967,841 | 83,109,782 |
| **across all `ADJ`** | 141,953 | 83,142,390 | 83,284,343 |

### Observed Contingencies for *fundamentally* + *critical*

|    | *critical* | $\neg$ *critical* | across all `ADV` |
|---:|---:|---:|---:|
| ***fundamentally*** | 46 | 40,463 | 40,509 |
| **$\neg$ *fundamentally*** | 141,907 | 83,101,927 | 83,243,834 |
| **across all `ADJ`** | 141,953 | 83,142,390 | 83,284,343 |

### Observed Contingencies for *extremely* + *critical*

|    | *critical* | $\neg$ *critical* | across all `ADV` |
|---:|---:|---:|---:|
| ***extremely*** | 2,130 | 974,033 | 976,163 |
| **$\neg$ *extremely*** | 139,823 | 82,168,357 | 82,308,180 |
| **across all `ADJ`** | 141,953 | 83,142,390 | 83,284,343 |

### Observed Contingencies for *nearly* + *affordable*

|    | *afforda

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
dtype: object

### Observed Contingencies for *nearly* + *critical*

|    | *critical* | $\neg$ *critical* | across all `ADV` |
|---:|---:|---:|---:|
| ***nearly*** | 12 | 174,549 | 174,561 |
| **$\neg$ *nearly*** | 141,941 | 82,967,841 | 83,109,782 |
| **across all `ADJ`** | 141,953 | 83,142,390 | 83,284,343 |

### Observed Contingencies for *fundamentally* + *critical*

|    | *critical* | $\neg$ *critical* | across all `ADV` |
|---:|---:|---:|---:|
| ***fundamentally*** | 46 | 40,463 | 40,509 |
| **$\neg$ *fundamentally*** | 141,907 | 83,101,927 | 83,243,834 |
| **across all `ADJ`** | 141,953 | 83,142,390 | 83,284,343 |

### Observed Contingencies for *extremely* + *critical*

|    | *critical* | $\neg$ *critical* | across all `ADV` |
|---:|---:|---:|---:|
| ***extremely*** | 2,130 | 974,033 | 976,163 |
| **$\neg$ *extremely*** | 139,823 | 82,168,357 | 82,308,180 |
| **across all `ADJ`** | 141,953 | 83,142,390 | 83,284,343 |

### Observed Contingencies for *nearly* + *affordable*

|    | *affordable* | $\neg$ *affordable* | across all `ADV` |
|---:|---:|---:|---:|
| ***nearly*** | 8 | 174,553 | 174,561 |
| **$\neg$ *nearly*** | 126,065 | 82,983,717 | 83,109,782 |
| **across all `ADJ`** | 126,073 | 83,158,270 | 83,284,343 |

### Observed Contingencies for *fundamentally* + *affordable*

|    | *affordable* | $\neg$ *affordable* | across all `ADV` |
|---:|---:|---:|---:|
| ***fundamentally*** | 2 | 40,507 | 40,509 |
| **$\neg$ *fundamentally*** | 126,071 | 83,117,763 | 83,243,834 |
| **across all `ADJ`** | 126,073 | 83,158,270 | 83,284,343 |

### Observed Contingencies for *extremely* + *affordable*

|    | *affordable* | $\neg$ *affordable* | across all `ADV` |
|---:|---:|---:|---:|
| ***extremely*** | 2,253 | 973,910 | 976,163 |
| **$\neg$ *extremely*** | 123,820 | 82,184,360 | 82,308,180 |
| **across all `ADJ`** | 126,073 | 83,158,270 | 83,284,343 |

### Observed Contingencies for *nearly* + *healthy*

|    | *healthy* | $\neg$ *healthy* | across all `ADV` |
|---:|---:|---:|---:|
| ***nearly*** | 14 | 174,547 | 174,561 |
| **$\neg$ *nearly*** | 113,052 | 82,996,730 | 83,109,782 |
| **across all `ADJ`** | 113,066 | 83,171,277 | 83,284,343 |

### Observed Contingencies for *fundamentally* + *healthy*

|    | *healthy* | $\neg$ *healthy* | across all `ADV` |
|---:|---:|---:|---:|
| ***fundamentally*** | 114 | 40,395 | 40,509 |
| **$\neg$ *fundamentally*** | 112,952 | 83,130,882 | 83,243,834 |
| **across all `ADJ`** | 113,066 | 83,171,277 | 83,284,343 |

### Observed Contingencies for *extremely* + *healthy*

|    | *healthy* | $\neg$ *healthy* | across all `ADV` |
|---:|---:|---:|---:|
| ***extremely*** | 1,223 | 974,940 | 976,163 |
| **$\neg$ *extremely*** | 111,843 | 82,196,337 | 82,308,180 |
| **across all `ADJ`** | 113,066 | 83,171,277 | 83,284,343 |
